In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd
import random

In [2]:
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjac/_search" # url do TJ
api_key = "APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==" # Chave pública

#payload = json.dumps({
#  "size": 1000,
#  "query": {
#    "match": {"classe.codigo": 1268} #maria da penha 
#  },
#  "sort": [{"dataAjuizamento": {"order": "desc"}}] # ou asc
#})
#from_index=0
#payload = json.dumps({
#    "size": 1000,
 #   "from": from_index,
  #  "query": {
   #     "match": {"movimentos.nome": "sentença"},
    #    #"match": {"classe.codigo": 1116} #execução fiscal
     #   #"match": {"classe.codigo": 92} #despejo
      #  #"match": {"classe.codigo": 1268} #maria da penha
       # "match": {"classe.codigo": 7},
       # "match": {"assuntos.codigo": [7779, 10433]}
#    },
#    "sort": [{"dataAjuizamento": {"order": "desc"}}]
#}
#)


from_index = 0
payload = json.dumps({
    "size": 1000,
    "from": from_index,
    "query": {
        "bool": {
            "must": [
                {"match": {"movimentos.nome": "sentença"}},
                #{"term": {"classe.codigo": 7}},
                #{"terms": {"assuntos.codigo": [7779, 10433]}}
            ]
        }
    },
    "sort": [{"dataAjuizamento": {"order": "desc"}}]
})

headers = {
  'Authorization': api_key,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)  # <Response [200]>
dados_dict = response.json() # <class 'dict'>
print(len(dados_dict))

4


In [47]:
display(dados_dict['hits']['hits'][999]) # Exemplo

{'_index': 'api_publica_tjac',
 '_id': 'TJAC_JE_07073175120248010070',
 '_score': None,
 '_source': {'classe': {'codigo': 12078,
   'nome': 'Cumprimento de Sentença contra a Fazenda Pública'},
  'numeroProcesso': '07073175120248010070',
  'sistema': {'codigo': 3, 'nome': 'SAJ'},
  'formato': {'codigo': 1, 'nome': 'Eletrônico'},
  'tribunal': 'TJAC',
  'dataHoraUltimaAtualizacao': '2025-07-09T03:04:23.209Z',
  'grau': 'JE',
  '@timestamp': '2025-10-05T09:21:41.135368693Z',
  'dataAjuizamento': '20241127171241',
  'movimentos': [{'complementosTabelados': [{'codigo': 4,
      'valor': 107,
      'nome': 'Certidão',
      'descricao': 'tipo_de_documento'}],
    'codigo': 60,
    'nome': 'Expedição de documento',
    'dataHora': '2025-06-26T07:10:05.000Z'},
   {'codigo': 246,
    'nome': 'Definitivo',
    'dataHora': '2025-06-30T10:02:28.000Z'},
   {'complementosTabelados': [{'codigo': 4,
      'valor': 73,
      'nome': 'Alvará',
      'descricao': 'tipo_de_documento'}],
    'codigo': 60,


In [56]:
# Query para pegar classes com nome

pd.set_option('display.max_rows', None)

payload = json.dumps({
    "size": 10000,
    "query": {"match_all": {}},
    "_source": ["classe.codigo", "classe.nome"]
})

response = requests.post(url, headers=headers, data=payload)
dados = response.json()

# Contar códigos e nomes
classes_lista = []
for hit in dados['hits']['hits']:
    classe = hit['_source'].get('classe', {})
    if classe:
        classes_lista.append({
            'codigo': classe.get('codigo'),
            'nome': classe.get('nome')
        })

df = pd.DataFrame(classes_lista)
contagem = df.groupby(['codigo', 'nome']).size().reset_index(name='quantidade')
contagem = contagem.sort_values('quantidade', ascending=False)

print(contagem)

    codigo                                               nome  quantidade
22     436             Procedimento do Juizado Especial Cível        3168
23     460                            Recurso Inominado Cível        2091
3      156                            Cumprimento de sentença        1720
12     278                              Termo Circunstanciado         806
32   14695  Procedimento do Juizado Especial da Fazenda Pú...         658
28   12078   Cumprimento de Sentença contra a Fazenda Pública         601
30   12154                   Execução de Título Extrajudicial         264
20     355                          Carta Precatória Criminal         178
26   10944              Ação Penal - Procedimento Sumaríssimo         144
21     417                                  Apelação Criminal          50
11     272            Representação Criminal/Notícia de Crime          49
10     261                             Carta Precatória Cível          36
7      202                            

In [59]:
import requests
import json

url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjac/_search"
api_key = "APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw=="

payload = json.dumps({
    "size": 300,
    "query": {
        "bool": {
            "must": [
                {"term": {"classe.codigo": 436}},
                {"terms": {"assuntos.codigo": [10433, 7779]}}
            ]
        }
    }
})

headers = {
    'Authorization': api_key,
    'Content-Type': 'application/json'
}

response = requests.post(url, headers=headers, data=payload)
dados = response.json()

print(f"Total encontrado: {dados['hits']['total']['value']}")
print(f"Retornados: {len(dados['hits']['hits'])}")

# Salvar para processar
dados_dict = dados

# Ver primeiro processo
if len(dados['hits']['hits']) > 0:
    primeiro = dados['hits']['hits'][0]['_source']
    print(f"\nPrimeiro processo: {primeiro['numeroProcesso']}")
    print(f"Assuntos: {[a['nome'] for a in primeiro.get('assuntos', [])]}")

Total encontrado: 10000
Retornados: 300

Primeiro processo: 00021737420238010070
Assuntos: ['Indenização por Dano Moral']


In [63]:
# Ver quantos processos são do foro 1 (Rio Branco)
import requests
import json

url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjac/_search"
api_key = "APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw=="

# Buscar com filtro de município (Rio Branco = código IBGE 1200401)
payload = json.dumps({
    "size": 0,
    "query": {
        "bool": {
            "must": [
                {"term": {"classe.codigo": 436}},
                {"terms": {"assuntos.codigo": [10433, 7779]}},
                {"term": {"orgaoJulgador.codigoMunicipioIBGE": 1200401}}  # Rio Branco
            ]
        }
    }
})

headers = {
    'Authorization': api_key,
    'Content-Type': 'application/json'
}

response = requests.post(url, headers=headers, data=payload)
dados = response.json()

print(f"Total de processos em Rio Branco (foro 1): {dados['hits']['total']['value']}")

Total de processos em Rio Branco (foro 1): 4328


In [131]:
#verificar quantidade de sub assuntos
import requests
import json
from collections import Counter

url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjsc/_search"
api_key = "APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw=="

payload = json.dumps({
    "size": 1000,
    "query": {
        "term": {"classe.codigo": 7}
    },
    "_source": ["assuntos"]
})

headers = {
    'Authorization': api_key,
    'Content-Type': 'application/json'
}

response = requests.post(url, headers=headers, data=payload)
dados = response.json()

# Contar assuntos
assuntos_lista = []
for hit in dados['hits']['hits']:
    assuntos = hit['_source'].get('assuntos', [])
    if isinstance(assuntos, list):
        for assunto in assuntos:
            if isinstance(assunto, dict):
                assuntos_lista.append({
                    'codigo': assunto.get('codigo'),
                    'nome': assunto.get('nome')
                })

# Contar e ordenar
contagem = Counter((a['codigo'], a['nome']) for a in assuntos_lista)
ordenado = sorted(contagem.items(), key=lambda x: x[1], reverse=True)

# Printar
for (codigo, nome), qtd in ordenado:
    print(f"{codigo} - {nome}: {qtd}")

11806 - Empréstimo consignado: 124
9607 - Contratos Bancários: 87
11811 - Práticas Abusivas: 70
7779 - Indenização por Dano Moral: 67
10433 - Indenização por Dano Moral: 62
11974 - Cláusulas Abusivas: 58
7780 - Indenização por Dano Material: 47
6107 - Auxílio-Acidente (Art. 86): 45
7770 - Interpretação / Revisão de Contrato: 37
9596 - Prestação de Serviços: 33
10439 - Indenização por Dano Material: 33
10536 - Taxa de Coleta de Lixo: 32
7772 - Cartão de Crédito: 32
9597 - Seguro: 30
7621 - Seguro: 22
8843 - Assistência Judiciária Gratuita: 22
10441 - Acidente de Trânsito: 20
7757 - Auxílio-Doença Acidentário: 19
6095 - Aposentadoria por Incapacidade Permanente: 19
11810 - Dever de Informação: 18
7691 - Inadimplemento: 18
7698 - Perdas e Danos: 17
6101 - Auxílio por Incapacidade Temporária: 15
10467 - Despesas Condominiais: 14
10567 - Aposentadoria por Invalidez Acidentária: 12
7760 - Fornecimento de Energia Elétrica: 12
10582 - Rescisão / Resolução: 11
9587 - Compra e Venda: 11
11807 - 

In [153]:
# Ver todos os campos de um processo
processo_exemplo = dados_dict['hits']['hits'][0]['_source']

print("=== CAMPOS DISPONÍVEIS ===")
for campo in processo_exemplo.keys():
    print(f"\n{campo}: {processo_exemplo[campo]}")

=== CAMPOS DISPONÍVEIS ===

numeroProcesso: 50004234820228240004

classe: {'codigo': 7, 'nome': 'Procedimento Comum Cível'}

sistema: {'codigo': 4, 'nome': 'EPROC'}

formato: {'codigo': 1, 'nome': 'Eletrônico'}

tribunal: TJSC

dataHoraUltimaAtualizacao: 2025-06-14T12:18:11.348Z

grau: G1

@timestamp: 2025-06-16T00:20:09.900546044Z

dataAjuizamento: 6022-01-20T15:20:21.000Z

movimentos: [{'complementosTabelados': [{'codigo': 4, 'valor': 116, 'nome': 'Informações', 'descricao': 'tipo_de_documento'}], 'codigo': 581, 'nome': 'Documento', 'dataHora': '2023-02-17T07:52:33.000Z'}, {'complementosTabelados': [{'codigo': 18, 'valor': 40, 'nome': 'outros motivos', 'descricao': 'motivo_da_remessa'}], 'codigo': 123, 'nome': 'Remessa', 'dataHora': '2023-02-16T15:58:55.000Z'}, {'codigo': 848, 'nome': 'Trânsito em julgado', 'dataHora': '2023-02-16T15:58:47.000Z'}, {'codigo': 1051, 'nome': 'Decurso de Prazo', 'dataHora': '2023-02-16T01:10:19.000Z'}, {'codigo': 1051, 'nome': 'Decurso de Prazo', 'dataHo

In [ ]:
import os
import glob
import pandas as pd
from IPython.display import clear_output
import time

# Executar em loop
for _ in range(100):  # 100 iterações
    clear_output(wait=True)
    
    # Contar progresso
    htmls = len(glob.glob('arquivos/*.html'))
    backups = sorted(glob.glob('backup_tjac_*.csv'))
    
    print("=== MONITORAMENTO SCRAPING ===")
    print(f"HTMLs salvos: {htmls}/1200")
    print(f"Progresso: {(htmls/1200)*100:.1f}%")
    
    if backups:
        ultimo = backups[-1]
        df = pd.read_csv(ultimo)
        print(f"\nÚltimo backup: {ultimo}")
        print(f"Processos: {len(df)}")
        print(f"Com sentença: {df['procedente'].notna().sum()}")
    
    # Verificar se terminou
    if os.path.exists('processos_tjac_final.csv'):
        print("\n✓✓✓ CONCLUÍDO! ✓✓✓")
        df_final = pd.read_csv('processos_tjac_final.csv')
        print(f"Total: {len(df_final)}")
        print(f"Com sentença: {df_final['procedente'].notna().sum()}")
        break
    
    time.sleep(30)  # Atualizar a cada 30 segundosfrom selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd
import random
import os
import requests
import json

def formatar_numero_processo(numero):
    return f"{numero[:7]}-{numero[7:9]}.{numero[9:13]}.{numero[13]}.{numero[14:16]}.{numero[16:]}"

def consultar_processo_tjac(numero_processo, foro="1"):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless=new')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    try:
        driver = webdriver.Chrome(options=options)
        numero_formatado = formatar_numero_processo(numero_processo)
        url = f"https://esaj.tjac.jus.br/cpopg/show.do?processo.codigo=01000EPOS0000&processo.foro={foro}&processo.numero={numero_formatado}"
        
        driver.get(url)
        time.sleep(3)
        
        try:
            botoes = driver.find_elements(By.LINK_TEXT, "Mais")
            for botao in botoes:
                try:
                    driver.execute_script("arguments[0].click();", botao)
                    time.sleep(2)
                except:
                    pass
        except:
            pass
        
        html = driver.page_source
        driver.quit()
        return html, url
    except Exception as e:
        print(f"Erro: {e}")
        return None, None

def extrair_dados(html, numero_processo, url_processo):
    soup = BeautifulSoup(html, 'html.parser')
    
    dados = {
        'num_processo': numero_processo,
        'juiz': None,
        'procedente': None,
        'desc': None,
        'link_processo': url_processo,
        'link_pdf': None
    }
    
    texto = soup.get_text()
    linhas_texto = texto.split('\n')
    
    # Juiz
    for i, linha in enumerate(linhas_texto):
        if 'juiz' in linha.lower() and i + 1 < len(linhas_texto):
            proximo = linhas_texto[i + 1].strip()
            if proximo and len(proximo) > 5:
                dados['juiz'] = proximo
                break
    
    # Termos
    termos_parcial = [
        'julgo procedente em parte', 'julgo procedentes em parte',
        'julgado procedente em parte', 'julgados procedentes em parte',
        'julgo parcialmente procedente', 'julgado parcialmente procedente'
    ]
    
    termos_improcedente = [
        'julgo improcedente', 'julgo improcedentes',
        'julgado improcedente', 'julgados improcedentes'
    ]
    
    termos_procedente = [
        'julgo procedente', 'julgo procedentes',
        'julgado procedente', 'julgados procedentes'
    ]
    
    linhas_tabela = soup.find_all('tr')
    
    for linha in linhas_tabela:
        texto_linha = linha.get_text()
        texto_linha_lower = texto_linha.lower()
        
        resultado_encontrado = None
        
        if any(termo in texto_linha_lower for termo in termos_parcial):
            resultado_encontrado = 2
        elif any(termo in texto_linha_lower for termo in termos_improcedente):
            resultado_encontrado = 0
        elif any(termo in texto_linha_lower for termo in termos_procedente):
            if not any(termo in texto_linha_lower for termo in termos_improcedente):
                resultado_encontrado = 1
        
        if resultado_encontrado is not None:
            link = linha.find('a', href=True)
            
            if link or dados['procedente'] is None:
                dados['procedente'] = resultado_encontrado
                dados['desc'] = texto_linha.strip()
                
                if link:
                    dados['link_pdf'] = 'https://esaj.tjac.jus.br' + link['href']
                    break
    
    return dados

def salvar_html(numero_processo, html):
    if not os.path.exists('arquivos'):
        os.makedirs('arquivos')
    
    with open(f'arquivos/{numero_processo}.html', 'w', encoding='utf-8') as f:
        f.write(html)

# Buscar 1200 processos do DataJud
print("Buscando 1200 processos no DataJud...")
url_api = "https://api-publica.datajud.cnj.jus.br/api_publica_tjac/_search"
api_key = "APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw=="

todos_processos = []

for pagina in range(6):  # 6 páginas de 200 = 1200
    payload = json.dumps({
        "size": 200,
        "from": pagina * 200,
        "query": {
            "bool": {
                "must": [
                    {"term": {"classe.codigo": 436}},
                    {"terms": {"assuntos.codigo": [10433, 7779]}}
                ]
            }
        },
        "_source": ["numeroProcesso"]
    })

    headers = {
        'Authorization': api_key,
        'Content-Type': 'application/json'
    }

    response = requests.post(url_api, headers=headers, data=payload)
    dados_api = response.json()
    todos_processos.extend(dados_api['hits']['hits'])
    print(f"  Página {pagina+1}: {len(todos_processos)} processos acumulados")

processos = [hit['_source']['numeroProcesso'] for hit in todos_processos]
print(f"\n✓ Total: {len(processos)} processos para processar\n")
print("="*60)

# Processar 1200
resultados = []

for i, processo in enumerate(processos):
    print(f"\n[{i+1}/{len(processos)}] {processo}")
    
    html, url = consultar_processo_tjac(processo)
    
    if html:
        dados = extrair_dados(html, processo, url)
        resultados.append(dados)
        salvar_html(processo, html)
        
        print(f"  Juiz: {dados['juiz']}")
        print(f"  Procedente: {dados['procedente']}")
    else:
        resultados.append({
            'num_processo': processo,
            'juiz': None,
            'procedente': None,
            'desc': None,
            'link_processo': None,
            'link_pdf': None
        })
    
    time.sleep(random.uniform(4, 7))
    
    # Backup a cada 50
    if (i + 1) % 50 == 0:
        df = pd.DataFrame(resultados)
        df.to_csv(f'backup_tjac_{i+1}.csv', index=False, encoding='utf-8-sig')
        print(f"  ✓✓ BACKUP: backup_tjac_{i+1}.csv")

# Salvar final
df_final = pd.DataFrame(resultados)
df_final.to_csv('processos_tjac_final.csv', index=False, encoding='utf-8-sig')

print("\n" + "="*60)
print("=== RESUMO FINAL ===")
print(f"Total processado: {len(df_final)}")
print(f"Com juiz: {df_final['juiz'].notna().sum()}")
print(f"Com sentença: {df_final['procedente'].notna().sum()}")
print(f"Procedentes (1): {(df_final['procedente'] == 1).sum()}")
print(f"Improcedentes (0): {(df_final['procedente'] == 0).sum()}")
print(f"Parciais (2): {(df_final['procedente'] == 2).sum()}")
print(f"\n✓ CSV final: processos_tjac_final.csv")
print(f"✓ HTMLs: ./arquivos/")
print("="*60)

Buscando 1200 processos no DataJud...
  Página 1: 200 processos acumulados
  Página 2: 400 processos acumulados
  Página 3: 600 processos acumulados
  Página 4: 800 processos acumulados
  Página 5: 1000 processos acumulados
  Página 6: 1200 processos acumulados

✓ Total: 1200 processos para processar


[1/1200] 00021737420238010070
  Juiz: Evelin Campos Cerqueira Bueno
  Procedente: 0
